# This notebook implements the BILP-Q tutorial in Cirq

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq



In [46]:
import cirq_google
from Utils_CSG import *
import sympy 
import numpy as np
from typing import Sequence, Tuple

## Starting with the a given CSG
Using the same steps as in the BILP-Q Qiskit tutorial to generate the Q-matrix

In [2]:
coalition_values={
    '1':30,
    '2':40,
    '3':25,
    '1,2':70,
    '1,3':60,
    '2,3':65,
    '1,2,3':90
}
coalition_values

{'1': 30, '2': 40, '3': 25, '1,2': 70, '1,3': 60, '2,3': 65, '1,2,3': 90}

In [3]:
c,S,b = convert_to_BILP(coalition_values)       # A function in Utils_CSG.py
print(f'c = {c}\nS = {S}\nb = {b}')

c = [30, 40, 25, 70, 60, 65, 90]
S = [[1, 0, 0, 1, 1, 0, 1], [0, 1, 0, 1, 0, 1, 1], [0, 0, 1, 0, 1, 1, 1]]
b = [1, 1, 1]


In [4]:
qubo_penalty = 50 * -1

linear,quadratic = get_QUBO_coeffs(c,S,b,qubo_penalty)        # A function in Utils_CSG.py

print(f"Linear Coefficients = {linear} \n Quadratic Coefficients = {quadratic}")

Linear Coefficients = {'x_6': -240.0, 'x_3': -170.0, 'x_4': -160.0, 'x_5': -165.0, 'x_0': -80.0, 'x_1': -90.0, 'x_2': -75.0} 
 Quadratic Coefficients = {('x_3', 'x_6'): 200.0, ('x_4', 'x_6'): 200.0, ('x_5', 'x_6'): 200.0, ('x_0', 'x_3'): 100.0, ('x_0', 'x_4'): 100.0, ('x_0', 'x_6'): 100.0, ('x_1', 'x_3'): 100.0, ('x_1', 'x_5'): 100.0, ('x_1', 'x_6'): 100.0, ('x_2', 'x_4'): 100.0, ('x_2', 'x_5'): 100.0, ('x_2', 'x_6'): 100.0, ('x_3', 'x_4'): 100.0, ('x_3', 'x_5'): 100.0, ('x_4', 'x_5'): 100.0}


In [40]:
Q = np.zeros([len(linear),len(linear)])

#diagonal elements
for key,value in linear.items():
    Q[int(key.split('_')[1]),int(key.split('_')[1])] = value

#non diagonal elements
for key,value in quadratic.items():
    Q[int(key[0].split('_')[1]),int(key[1].split('_')[1])] = value/2
    Q[int(key[1].split('_')[1]),int(key[0].split('_')[1])] = value/2

Q.round(1)

array([[ -80.,    0.,    0.,   50.,   50.,    0.,   50.],
       [   0.,  -90.,    0.,   50.,    0.,   50.,   50.],
       [   0.,    0.,  -75.,    0.,   50.,   50.,   50.],
       [  50.,   50.,    0., -170.,   50.,   50.,  100.],
       [  50.,    0.,   50.,   50., -160.,   50.,  100.],
       [   0.,   50.,   50.,   50.,   50., -165.,  100.],
       [  50.,   50.,   50.,  100.,  100.,  100., -240.]])

## Initializing the circuit

There are many other ways to create circuts. Worth mentioning is that cirq can also create grid-circuits and check, whether all two-qubits gates are only applied to adjacent qubits. This can be usefull when implementing for real quantum devices (D-Wave part).

In [69]:
problem_size = len(c) # number of possible coalitions -1 = number of qubits needed

qaoa = cirq.Circuit()
qaoa.append(cirq.H(q) for q in cirq.LineQubit.range(problem_size))
print(qaoa)



0: ───H───

1: ───H───

2: ───H───

3: ───H───

4: ───H───

5: ───H───

6: ───H───


## Setting the depth of qaoa and introducing parameters
Cirq uses its own class of symbols as variables.

In [70]:
p = 3 # number of layers

beta = [sympy.Symbol("β_"+str(i)) for i in range(p)]
gamma = [sympy.Symbol("γ_"+str(i)) for i in range(p)]
print(beta, gamma)
    

[β_0, β_1, β_2] [γ_0, γ_1, γ_2]


## Creating the Hamiltonians for mixer and cost unitaries

Adding operations to a circuit is usually done by appending
To add a unitary to the circuit we have to define an ordered list of operations that corresponds to that unitary

In [71]:
# the mixer layer should just be a list of X rotation gates with parameter beta on each qubit

def mixer(circuit, beta_value: float):
    n_qubits = len(circuit.get_independent_qubit_sets()) 
    circuit.append(cirq.X(q) ** beta_value for q in cirq.LineQubit.range(n_qubits))
    return circuit
        


In [72]:
#testing
qaoa = mixer(qaoa, beta[0])
print(qaoa)

0: ───H───X^(β_0)───

1: ───H───X^(β_0)───

2: ───H───X^(β_0)───

3: ───H───X^(β_0)───

4: ───H───X^(β_0)───

5: ───H───X^(β_0)───

6: ───H───X^(β_0)───
